# Austrian energy system Tutorial Part 3: A Policy Scenario

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Austrian energy system baseline scenario (``austria.ipynb``) and solved it successfully

**Introduction**

In this notebook, we will step through creating and investigating a policy scenario.

We will look into the effect of wind generation subsidies on the electricity sector. These subsidies will take the form of capital investment subsidies, effectively reducing the costs of capital investment.

# Setup and Helper Variables

In [ ]:
import ixmp as ix

# launch the IX modeling platform using the local default database
mp = ix.Platform()

In [ ]:
country = "Austria"

plants = [
    "coal_ppl",
    "gas_ppl",
    "oil_ppl",
    "bio_ppl",
    "hydro_ppl",
    "wind_ppl",
    "solar_pv_ppl",  # actually primary -> final
]

lights = [
    "bulb",
    "cfl",
]

# Create the New Policy Scenario Datastructure

This operation is done by cloning the baseline scenario, which will then be edited.

In [ ]:
scen_df = (
    mp.scenario_list()
)  # to find the name of the wanted scenario, simply print scen_df
scen_df.loc[
    scen_df["scenario"] == "baseline"
]  # to show just the scenario we are interested in

In [ ]:
import message_ix

model = "Austrian energy model"
scen = "baseline"
mp.open_db()  # only open if you're on a local database!
base = message_ix.Scenario(mp, model, scen)
scenario = base.clone(
    model, "wind subsidies", "testing wind subsidies", keep_solution=False
)
scenario.check_out()

In [ ]:
horizon = scenario.set("year")

# Edit the Datastructure with the Scenario Parameters

Introduce subsidies that decay with time. Initially they are 50% of the total investment costs and decay to 0.

In [ ]:
inv_cost_df = scenario.par("inv_cost")
inv_cost_df.loc[inv_cost_df["technology"] == "wind_ppl"]

In [ ]:
import numpy as np
import pandas as pd

subsidies = np.array([0.5, 0.5, 0.75, 1])
inv_cost = pd.DataFrame(
    {
        "node_loc": country,
        "year_vtg": horizon,
        "technology": "wind_ppl",
        "value": 1100 * subsidies,
        "unit": "USD/kW",
    }
)
scenario.add_par("inv_cost", inv_cost)

In [ ]:
scenario.par("inv_cost").tail()

# Solve the Model

In [ ]:
scenario.commit("wind subsidies added")
scenario.set_as_default()

In [ ]:
scenario.solve()

In [ ]:
scenario.var("OBJ")["lvl"]

# Investigate the Results

In [ ]:
import matplotlib.pyplot as plt

from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline
plt.style.use("ggplot")

config = dict(filters=dict(t=plants))
base_rep = Reporter.from_scenario(base, **config)
prepare_plots(base_rep)
scen_rep = Reporter.from_scenario(scenario, **config)
prepare_plots(scen_rep)

In [ ]:
base_rep.get("plot new capacity")
scen_rep.get("plot new capacity")

In [ ]:
base_rep.get("plot capacity")
scen_rep.get("plot capacity")

In [ ]:
base_rep.get("plot activity")
scen_rep.get("plot activity")

In [ ]:
mp.close_db()